In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from scipy.stats import multivariate_normal as mvnorm
from scipy.stats import uniform, invgamma, bernoulli, poisson, norm
import statsmodels.api as sm
import pandas as pd
from preprocessing import prepare_data, prepare_data_no_standardizing, MAR_data_deletion
from model_code import Gibbs_MH

In [3]:
df = pd.read_csv('student-mat.csv', sep=",")
features = ["age", "sex", "failures", "higher", "Medu", 
            "absences", "G2", "G3"]
df = df[features]
df.head()

,age,sex,failures,higher,Medu,absences,G2,G3
0,18,F,0,yes,4,6,6,6
1,17,F,0,yes,1,4,5,6
2,15,F,3,yes,1,10,8,10
3,15,F,0,yes,4,2,14,15
4,16,F,0,yes,3,4,10,10


In [4]:
X_df, y_df = prepare_data_no_standardizing(df)
X_df.head()

,age,failures,Medu,absences,G2,sex_M,higher_yes,intercept
0,18,0,4,6,6,0,1,1
1,17,0,1,4,5,0,1,1
2,15,3,1,10,8,0,1,1
3,15,0,4,2,14,0,1,1
4,16,0,3,4,10,0,1,1


In [5]:
X_df_missing = MAR_data_deletion(X_df, 0.05, 0.05, 'higher_yes', 'absences')
X_df_missing

,age,failures,Medu,absences,G2,sex_M,higher_yes,intercept
0,18,0,4,6.0,6,0,1.0,1
1,17,0,1,4.0,5,0,1.0,1
2,15,3,1,10.0,8,0,1.0,1
3,15,0,4,2.0,14,0,1.0,1
4,16,0,3,4.0,10,0,1.0,1
...,...,...,...,...,...,...,...,...
390,20,2,2,11.0,9,1,1.0,1
391,17,0,3,3.0,16,1,1.0,1
392,21,3,1,3.0,8,1,1.0,1
393,18,0,3,0.0,12,1,1.0,1


In [6]:
X = X_df.to_numpy()
y = y_df.to_numpy()
B = 10
n = len(y)
higher_yes_col = 6
absences_col = 3
tau = 1

In [8]:
(betas, sigmas2, higher_yes_sim, absences_sim, alphas0, alphas1, gammas0, gammas1,
 accepts_alpha0, accepts_alpha1, accepts_gamma0, accepts_gamma0) = Gibbs_MH(X, y, B, n, higher_yes_col, absences_col, tau)